# Construcción Curva OIS

Tenemos que:

$$
\begin{equation}
\Pi\left(t,X\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T\right)\Pi\left(T,X\right)\right]
\end{equation}
$$

$$
\begin{equation}
D_c\left(t,T\right)=\exp\left[-\int_t^Tr_c\left(u\right)du\right]
\end{equation}
$$

$$
\begin{equation}
\mathbb{E}_t^{Q_f}\left[D_c\left(t,T\right)\right]=P_c\left(t,T\right)
\end{equation}
$$

$$
\begin{equation}
D_c\left(t,T\right)={B_c\left(t,T\right)}^{-1}
\end{equation}
$$

Consideremos el valor de la pata fija de un OIS. Para fijar las ideas, supongamos una pata a 2Y con cupones anuales. El valor de la pata fija es:

$$
\begin{equation}
\Pi\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right)\cdot r\cdot\frac{T_1-T_0}{360}+D_c\left(t,T_2\right)\left(1+r \cdot \frac{T_2-T_1}{360}\right)\right]
\end{equation}
$$

$$
\begin{equation}
\Pi\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right)\right]\cdot r\cdot\frac{T_1-T_0}{360}+\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_2\right)\right]\left(1+r \cdot \frac{T_2-T_1}{360}\right)
\end{equation}
$$

Mientras que el valor de la pata flotante es:

$$
\begin{equation}
\Pi\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right) \cdot \left(\frac{B_c\left(t,T_1\right)}{B_c\left(t,T_0\right)}-1\right)\right]+\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_2\right) \cdot \frac{B_c\left(t,T_2\right)}{B_c\left(t,T_1\right)}\right]
\end{equation}
$$

$$
\begin{equation}
\Pi\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_0\right)-D_c\left(t,T_1\right)\right]+\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right)\right]
\end{equation}
$$

$$
\begin{equation}
\Pi\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_0\right)\right]
\end{equation}
$$

Finalmente, para $t=T_0$ tenemos que:

$$
\begin{equation}
\mathbb{E}_{t_0}^{Q_f}\left[D_c\left(t_0,T_1\right)\right]\cdot r\cdot\frac{T_1-T_0}{360}+\mathbb{E}_{t_0}^{Q_f}\left[D_c\left(t_0,T_2\right)\right]\left(1+r \cdot \frac{T_2-T_1}{360}\right)=1
\end{equation}
$$

O sea, el valor presente de los flujos de la pata fija es 1.

## Ejercicio

Demuestre que, para valorizar, el valor esperado en $t$ de un flujo entre $T_1$ y $T_2$ de la pata flotante se puede calcular como:

$$
nominal\cdot r_{T_1,T_2}\cdot\frac{T_2-T_1}{360}
$$

donde

$$
r_{T_1,T_2}=\left( \frac{P_c\left(t, T_1\right)}{P_c\left(t, T_2\right)}-1\right)\cdot\frac{360}{T_2-T_1}
$$

In [1]:
# Pendiente

## Librerías

In [2]:
from finrisk import QC_Financial_3 as Qcf
from scipy.optimize import root_scalar
import modules.auxiliary as aux
import pandas as pd
import math

## Funciones

In [3]:
def make_fixed_leg(start_date: str, tenor: str, rate_value: float) -> Qcf.Leg:
    """
    Construye un pata fija con algunos parámetros prefijados:
    
    - recibo o pago: R
    - nocional: 1,000,000
    - moneda: USD
    - periodicidad: 1Y
    - business adjustment rule: MODFOLLOW
    - stub period: SHORTBACK
    
    params:
    
    - start_date: fecha inicial de la pata en formato ISO
    - tenor: plazo estructurado de la pata (1Y, 2Y, 18M, ...)
    - rate_value: valor de la tasa fija
    
    return:
    
    - objeto `Qcf.Leg` con cashflows de tipo `FixedRateCashflow`
    """
    # Recibo o pago los flujos de esta pata
    rp = Qcf.RecPay.RECEIVE

    # Periodicidad de pago
    periodicidad = Qcf.Tenor('1Y')

    # Tipo de período irregular (si lo hay)
    periodo_irregular = Qcf.StubPeriod.SHORTBACK

    # Regla para ajustes de días feriados
    bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW

    # Número de días después de la fecha final de devengo en que se paga el flujo
    lag_pago = 0

    # Nocional del contrato
    nocional = 10000000.0

    # Considera amortización
    amort_es_flujo = True

    # Moneda
    moneda = Qcf.QCUSD()

    # Es un bono de RF
    es_bono = False

    # Fecha de inicio de devengo del primer cupón
    fecha_inicial = Qcf.build_qcdate_from_string(start_date)

    # Fecha final de devengo, antes de ajustes, del último cupón
    qc_tenor = Qcf.Tenor(tenor)
    meses = qc_tenor.get_years()*12 + qc_tenor.get_months()
    if meses > 0:
        fecha_final = fecha_inicial.add_months(meses)
    else:
        dias = qc_tenor.get_days()
        fecha_final = fecha_inicial.add_days(dias)

    valor_tasa = rate_value
    tasa_cupon = Qcf.QCInterestRate(
        valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    bus_cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)

    # Se da de alta el objeto y se retorna
    return Qcf.LegFactory.build_bullet_fixed_rate_leg(
        rp,
        fecha_inicial,
        fecha_final,
        bus_adj_rule,
        periodicidad,
        periodo_irregular,
        bus_cal,
        lag_pago,
        nocional,
        amort_es_flujo,
        tasa_cupon,
        moneda,
        es_bono)

In [4]:
def error_with_rate(rate: float, *args) -> float:
    """
    Calcula el error entre el valor presente de una pata fija calculado con una curva cero y el nocional
    de la pata fija. Esta función está diseñada para servir de apoyo al proceso de bootstrapping de una curva
    cero cupón de OIS.
    
    params:
    
    - rate: valor a utilizar para la última tasa de la curva
    - args: iterable con plazos de la curva, tasas de la curva, objeto Qcf.Leg a valorizar y fecha de
    valorización.
    
    return:
    
    - float con el monto del error.
    """
    # Para usar la librería, los plazos y tasas deben estar en estos formatos.
    # Recordar que la librería está escrita en C++. 
    plazos = Qcf.long_vec()
    for p in args[0]:
        plazos.append(p)
    
    tasas = Qcf.double_vec()
    for t in args[1]:
        tasas.append(t)
    
    pata = args[2]
    
    fecha_val = args[3]
    
    # Se construye una curva cero Qcf
    curva = Qcf.QCCurve(plazos, tasas)
    cuantos = curva.get_length()
    par = curva.get_values_at(cuantos - 1)
    curva.set_pair(par.tenor, rate) # La variable rate se transforma en la última tasa de la curva
    curva = Qcf.QCLinearInterpolator(curva)
    curva = Qcf.ZeroCouponCurve(curva, Qcf.QCInterestRate(
        0.0, Qcf.QCAct365(), Qcf.QCContinousWf()))
    
    # Se da de alta el objeto que calcula valores presentes
    vp = Qcf.PresentValue()
    
    # Se retorna la diferencia entre el vp calculado con la curva y el nominal
    return vp.pv(fecha_val, pata, curva) - pata.get_cashflow_at(0).get_nominal()

## Formatos `DataFrame`

In [5]:
frmt = {'rate_value': '{:.8%}', 'valor_tasa': '{:.8%}',
        'nominal': '{:,.2f}', 'amortizacion': '{:,.2f}',
        'interes': '{:,.2f}', 'flujo': '{:,.2f}'
       }

## Obtiene Data

In [6]:
swaps = pd.read_excel('data/20201012_sofr_zero.xlsx', sheet_name='upload')
swaps['start_date'] = swaps['start_date'].astype(str).str[:10]

In [7]:
swaps.style.format(frmt)

,start_date,tenor,rate_value,rate_type,periodicity,stub_period
0,2020-10-14,1D,0.08000000%,LINACT360,1Y,SHORTBACK
1,2020-10-14,7D,0.08290000%,LINACT360,1Y,SHORTBACK
2,2020-10-14,14D,0.07690000%,LINACT360,1Y,SHORTBACK
3,2020-10-14,21D,0.07630005%,LINACT360,1Y,SHORTBACK
4,2020-10-14,1M,0.07700000%,LINACT360,1Y,SHORTBACK
5,2020-10-14,2M,0.07700000%,LINACT360,1Y,SHORTBACK
6,2020-10-14,3M,0.08000000%,LINACT360,1Y,SHORTBACK
7,2020-10-14,4M,0.07700000%,LINACT360,1Y,SHORTBACK
8,2020-10-14,5M,0.07500000%,LINACT360,1Y,SHORTBACK
9,2020-10-14,6M,0.07400000%,LINACT360,1Y,SHORTBACK


## Construye Objetos `Qcf`

In [8]:
qc_swaps = []
for s in swaps.itertuples():
    leg = make_fixed_leg(s.start_date, s.tenor, s.rate_value)
    cuantos_flujos = leg.size()
    last_pmt_date = leg.get_cashflow_at(cuantos_flujos - 1).get_settlement_date()
    start_date = Qcf.build_qcdate_from_string(s.start_date)
    plazo = start_date.day_diff(last_pmt_date)
    qc_swaps.append((leg, plazo))

In [9]:
aux.show_leg(qc_swaps[16][0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"10,000,000.00",0.00,"6,417.92",True,"6,417.92",USD,0.06330000%,LinAct360
1,2021-10-14,2022-04-14,2022-04-14,"10,000,000.00","10,000,000.00","3,200.17",True,"10,003,200.17",USD,0.06330000%,LinAct360


## Construye Curva Cero Cupón

Demuestra función objetivo.

In [13]:
plazos = []
tasas = []
 
plazos.append(1) # Los plazos se calculan desde start_date (2020-10-14)
tasa = math.log(1 + .08 / 100 * 1 / 360)* 365.0 / 1
# 1 + .08% * 1/360 = exp(r * 1/365) -> log(1 + .08% * 1/360) * 365/1
tasas.append(tasa)

plazos.append(int(7))
tasas.append(.0)

error_with_rate(.0008, plazos, tasas, qc_swaps[1][0], Qcf.QCDate(14, 10, 2020))

7.768488567322493

In [14]:
plazos = []
tasas = []
 
plazos.append(1) # Los plazos se calculan desde start_date (2020-10-14)
tasa = math.log(1 + .08 / 100 * 1 / 360)* 365.0 / 1
# 1 + .08% * 1/360 = exp(r * 1/365) -> log(1 + .08% * 1/360) * 365/1
tasas.append(tasa)

for s in qc_swaps[1:]:
    plazos.append(s[1])
    tasas.append(0.0)
    
    x = root_scalar(
        error_with_rate,
        method='bisect',
        bracket=[0.0, .02],
        x0=.0008,
        args=(plazos, tasas, s[0], Qcf.QCDate(14, 10, 2020)),
        xtol=.00000000000000001
    )
    
    # print(x)
    # print()
    
    tasas[-1] = x.root

In [16]:
for_df = []
for p, t in zip(plazos, tasas):
    for_df.append((p,t))
df_curva = pd.DataFrame(for_df, columns=['plazo', 'tasa'])
df_curva['df'] = df_curva.apply(lambda row: math.exp(-row['plazo']*row['tasa']/365), axis=1)
df_curva.style.format({'tasa':'{:.4%}', 'df':'{:.4%}'})

,plazo,tasa,df
0,1,0.0811%,99.9998%
1,7,0.0841%,99.9984%
2,14,0.0780%,99.9970%
3,21,0.0774%,99.9955%
4,33,0.0781%,99.9929%
5,61,0.0781%,99.9870%
6,92,0.0811%,99.9796%
7,124,0.0781%,99.9735%
8,152,0.0760%,99.9683%
9,182,0.0750%,99.9626%


## Comprobación

Con el resultado (raíces) se construye un objeto `Qcf.ZeroCouponCurve`.

In [17]:
qc_plazos = Qcf.long_vec()
qc_tasas = Qcf.double_vec()
for p, t in zip(plazos, tasas):
    qc_plazos.append(p)
    qc_tasas.append(t)
curva_final = Qcf.QCCurve(qc_plazos, qc_tasas)
curva_final = Qcf.QCLinearInterpolator(curva_final)

# Se construye la curva
curva_final = Qcf.ZeroCouponCurve(
    curva_final,
    Qcf.QCInterestRate(0.0, Qcf.QCAct365(), Qcf.QCContinousWf())
)

Se calcula el valor presente de cada una de las patas fijas de los swaps.

In [18]:
vp = Qcf.PresentValue()
for i, s in enumerate(qc_swaps):
    pv = vp.pv(Qcf.QCDate(14, 10, 2020), s[0], curva_final)
    print(f'{i}: {pv:,.8f}')

0: 10,000,000.00000000
1: 10,000,000.00000000
2: 10,000,000.00000000
3: 10,000,000.00000000
4: 10,000,000.00000000
5: 10,000,000.00000000
6: 10,000,000.00000000
7: 10,000,000.00000000
8: 10,000,000.00000000
9: 10,000,000.00000000
10: 10,000,000.00000000
11: 10,000,000.00000000
12: 10,000,000.00000000
13: 10,000,000.00000000
14: 10,000,000.00000000
15: 10,000,000.00000000
16: 10,000,000.00000000
17: 10,000,000.00000000
18: 10,000,000.00000000
19: 10,000,000.00000000
20: 10,000,000.00000000
21: 10,000,000.00000000
22: 10,000,000.00000000
23: 10,000,000.00000000
24: 10,000,000.00000000
25: 10,000,000.00000000
26: 10,000,000.00000000
27: 10,000,000.00000000
28: 10,000,000.00000000
29: 10,000,000.00000000
30: 10,000,000.00000000
31: 10,000,000.00000000
32: 10,000,000.00000000
